In [1]:
import numpy as np
import nibabel as nib
import glob
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from tifffile import imsave

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [2]:
import nibabel as nib

In [24]:
SEGMENT_CLASSES = {
    0 : 'NOT tumor',   
    1 : 'NECROTIC/CORE', # or NON-ENHANCING tumor CORE - RED
    2 : 'EDEMA',  # Green
    3 : 'ENHANCING' # original 4 -> converted into 3 later, Yellow
}

In [3]:
t2_list = sorted(glob.glob(r'C:\Users\gk336\Downloads\brats\BraTS2020_ValidationData\MICCAI_BraTS2020_ValidationData//*/*t2.nii'))
t1ce_list = sorted(glob.glob(r'C:\Users\gk336\Downloads\brats\BraTS2020_ValidationData\MICCAI_BraTS2020_ValidationData/*/*t1ce.nii'))
flair_list = sorted(glob.glob(r'C:\Users\gk336\Downloads\brats\BraTS2020_ValidationData\MICCAI_BraTS2020_ValidationData/*/*flair.nii'))
mask_list = sorted(glob.glob(r'C:\Users\gk336\Downloads\brats\BraTS2020_ValidationData\MICCAI_BraTS2020_ValidationData/*/*seg.nii'))

In [4]:
len(t2_list)

125

In [5]:
for img in range(len(t2_list)):   #Using t1_list as all lists are of same size
    print("Now preparing image and masks number: ", img)
      
    temp_image_t2=nib.load(t2_list[img]).get_fdata()
    temp_image_t2=scaler.fit_transform(temp_image_t2.reshape(-1, temp_image_t2.shape[-1])).reshape(temp_image_t2.shape)
   
    temp_image_t1ce=nib.load(t1ce_list[img]).get_fdata()
    temp_image_t1ce=scaler.fit_transform(temp_image_t1ce.reshape(-1, temp_image_t1ce.shape[-1])).reshape(temp_image_t1ce.shape)
   
    temp_image_flair=nib.load(flair_list[img]).get_fdata()
    temp_image_flair=scaler.fit_transform(temp_image_flair.reshape(-1, temp_image_flair.shape[-1])).reshape(temp_image_flair.shape)
        
    temp_mask=nib.load(mask_list[img]).get_fdata()
    temp_mask=temp_mask.astype(np.uint8)
    temp_mask[temp_mask==4] = 3  #Reassign mask values 4 to 3
    #print(np.unique(temp_mask))
    
    
    temp_combined_images = np.stack([temp_image_flair, temp_image_t1ce, temp_image_t2], axis=3)
    temp_combined_images = np.stack([temp_image_flair, temp_image_t1ce, temp_image_t2], axis=3)
    
    #Crop to a size to be divisible by 64 so we can later extract 64x64x64 patches. 
    #cropping x, y, and z
    temp_combined_images=temp_combined_images[56:184, 56:184, 13:141]
    temp_mask = temp_mask[56:184, 56:184, 13:141]
    
    val, counts = np.unique(temp_mask, return_counts=True)
    
    if (1 - (counts[0]/counts.sum())) > 0.01:  #At least 1% useful volume with labels that are not 0
        print("Save Me")
        temp_mask= to_categorical(temp_mask, num_classes=4)
        np.save(r'C:\Users\gk336\Downloads\brats\training\train\\'+str(img)+'.npy', temp_combined_images)
        np.save(r'C:\Users\gk336\Downloads\brats\training\mask\\'+str(img)+'.npy', temp_mask)
        
    else:
        print("I am useless") 

Now preparing image and masks number:  0
Now preparing image and masks number:  1
Now preparing image and masks number:  2
Now preparing image and masks number:  3
Now preparing image and masks number:  4
Now preparing image and masks number:  5
Now preparing image and masks number:  6
Now preparing image and masks number:  7
Now preparing image and masks number:  8
Now preparing image and masks number:  9
Now preparing image and masks number:  10
Now preparing image and masks number:  11
Now preparing image and masks number:  12
Now preparing image and masks number:  13
Now preparing image and masks number:  14
Now preparing image and masks number:  15
Now preparing image and masks number:  16
Now preparing image and masks number:  17
Now preparing image and masks number:  18
Now preparing image and masks number:  19
Now preparing image and masks number:  20
Now preparing image and masks number:  21
Now preparing image and masks number:  22
Now preparing image and masks number:  23
No

In [41]:
  # or import split_folders

input_folder = r'C:\Users\gk336\Downloads\brats\training\\'
output_folder = r'C:\Users\gk336\Downloads\brats\Vaidation\\'
# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
import splitfolders

splitfolders.ratio(input_folder, # The location of dataset
                   output_folder, # The output location
                   seed=42, # The number of seed
                   ratio=(.8, .2), # The ratio of splited dataset
                   group_prefix=None, # If your dataset contains more than one file like ".jpg", ".pdf", etc
                   move=False # If you choose to move, turn this into True
                   )

In [13]:
# t2_list = sorted(glob.glob(r'C:\Users\gk336\OneDrive\Desktop\BrainTumor\archive (1)\MICCAI_BraTS_2018_Data_Validation/*/*t2.nii'))
# t1ce_list = sorted(glob.glob(r'C:\Users\gk336\OneDrive\Desktop\BrainTumor\archive (1)\MICCAI_BraTS_2018_Data_Validation/*/*t1ce.nii'))
# flair_list = sorted(glob.glob(r'C:\Users\gk336\OneDrive\Desktop\BrainTumor\archive (1)\MICCAI_BraTS_2018_Data_Validation/*/*flair.nii'))
# mask_list = sorted(glob.glob(r'C:\Users\gk336\OneDrive\Desktop\BrainTumor\archive (1)\MICCAI_BraTS_2018_Data_Validation/*/*seg.nii'))